# 导入包

In [2]:
import pandas as pd
import numpy as np

# 纳排标准：肺炎，＞18岁，1次ICU

In [ ]:
# 病毒
diagnoses = pd.read_csv("mimic-iv-2/hosp/diagnoses_icd.csv", header=0)
diagnoses_9e = diagnoses.loc[diagnoses["icd_code"].str.match("48[0,7,8]\d"), :]
print(diagnoses_9e.shape)
diagnoses_10e = pd.concat((diagnoses.loc[diagnoses["icd_code"].str.match("J1[0,1,2]\d"), :],
                           diagnoses.loc[diagnoses["icd_code"].str.match("J09\d"), :]),0) 
print(diagnoses_10e.shape)
diagnoses_pn = pd.concat((diagnoses_9e, diagnoses_10e), 0)
diagnoses_pn.to_csv("pn_virus/sofa_initial_diagnoses.csv", header=True, index=True)


In [3]:
diagnoses_bact = pd.read_csv("pn_bact/sofa_initial_diagnoses.csv")
diagnoses_vpn = diagnoses_pn.loc[~diagnoses_pn["hadm_id"].isin(set(diagnoses_bact["hadm_id"])),:]
diagnoses_vpn.to_csv("pn_virus/sofa_diagnoses.csv", header=True, index=True)
print(diagnoses_vpn.shape)

(2062, 5)


In [5]:
# 获取肺炎患者的stay信息并去除一次入院多次入住ICU的
patients = diagnoses_pn[["subject_id", "hadm_id"]].drop_duplicates()
icustay = pd.read_csv("mimic-iv-2/icu/icustays.csv", header=0)
patients_aicu = pd.merge(patients, icustay[["hadm_id","stay_id"]])
aicu_hadm = patients_aicu["hadm_id"].value_counts()[patients_aicu["hadm_id"].value_counts()==1]
patients_aicu =  patients_aicu.loc[patients_aicu["hadm_id"].isin(aicu_hadm.index),:]
patients_aicu.to_csv("pn_virus/patients_id.csv")

# 30天死亡分组

In [2]:
patients_id = pd.read_csv("pn_virus/patients_id.csv", header=0, index_col=0)
admission = pd.read_csv("mimic-iv-2/hosp/admissions.csv", header=0, index_col=0)
patients_oc = pd.merge(patients_id, admission[["hadm_id", "deathtime"]])

In [11]:
# icu stay
icustay = pd.read_csv("mimic-iv-2/icu/icustays.csv", header=0, index_col=0)
patients_icu_oc = pd.merge(patients_oc, icustay[["stay_id","intime"]])

# 计算入住ICU时间和死亡时间差
from datetime import datetime
death_days = []
for i in patients_icu_oc.index:
    if str(patients_icu_oc.loc[i,"deathtime"]) != "nan":
        death_day = (datetime.strptime(patients_icu_oc.loc[i,"deathtime"], "%Y-%m-%d %H:%M:%S") - 
                     datetime.strptime(patients_icu_oc.loc[i,"intime"], "%Y-%m-%d %H:%M:%S")).days
        death_days.append(death_day)
    else:
        death_days.append(31)
patients_icu_oc["death_day"] = death_days

In [14]:
# 分别保存30天内死亡患者
patients_icu_oc["30day_death"] = 0
patients_icu_oc.loc[patients_icu_oc["death_day"]<=30, "30day_death"] = 1
patients_icu_oc.loc[patients_icu_oc["death_day"]>30, "30day_death"] = 0
patients_icu_oc.loc[patients_icu_oc["death_day"]>30, "death_day"] = 31

patients_death = patients_icu_oc.loc[patients_icu_oc["30day_death"]==1,:]
patients_survival = patients_icu_oc.loc[patients_icu_oc["30day_death"]==0,:]

patients_death.to_csv("pn_virus/patients_death.csv")
patients_survival.to_csv("pn_virus/patients_survival.csv")
patients_icu_oc.to_csv("pn_virus/patients_oc.csv")